# The Life Modelling Problem

## Product Definition

Suppose we have an insurance product, in this case a simple term assurance which pays out S on death, on the condition that the policyholder pays a level premium of P per annum, and expires after T years.  We also incur initial expenses of I on setting up the policy, and renewal expenses of E per annum for the duration of the policy (this includes claim costs).

## Modelling

As actuaries, we want to calculate various metrics, for example to determine whether the policy is profitable, where to set the premium, and how much capital to hold.

These can generally be expressed as equations of value:

Expected Net Value = *E[PV(Premiums)] - E[PV(Claims)] - E[PV(Expenses)]*

Where *E[]* is the expectation (arithmetic mean), and PV is the present value function, summing over all t.

As *E[]* is linear, this can also be expressed as *E[PV(Premiums) - PV(Claims)]* or *PV[E(Premiums) - E(Claims)]*

## Model Assumptions and Structure

We can model this product as a Multi-State Model, with the states "In Force", "Claimed", "Lapsed" and "Expired", with one transition occuring per year.

There are 3 transitions in this model:

* In Force -> Claimed: q(t) - probability of dying between time t-1 and time t, and a claim being made
* In Force -> Lapsed: w(t) - the probability that the policyholder cancels the policy between time t-1 and time t
* In Force -> Expired: 1 if t is T, 0 otherwise

q(t) and w(t) will depend on the policyholder's state (age/sex etc), and also the time period.

For this simple example we will assume that q and w can be specified as tables:

In [45]:
q = {0: 0.001,
     1: 0.002,
     2: 0.003,
     3: 0.003,
     4: 0.004,
     5: 0.004,
     6: 0.005,
     7: 0.007,
     8: 0.009,
     9: 0.011}

w = {0: 0.05,
     1: 0.07,
     2: 0.08,
     3: 0.10,
     4: 0.14,
     5: 0.20,
     6: 0.20,
     7: 0.20,
     8: 0.10,
     9: 0.04}

We will also set values for P (Premium), S (Sum Assured / Claim Amount), T (Term); we ignore expenses (I and E set equal = 0)

In [46]:
P = 100
S = 25_000
T = 10

We can now define the expected numbers of policies in states/having events of interest:

- num_in_force: the number of policies in force
- num_death: the number of deaths 
- num_lapses: the number of lapses occuring

In [47]:
def num_in_force(t):
    if t == 0:
        return 1
    elif t >= T:
        return 0
    else:
        return num_in_force(t-1) - num_deaths(t-1) - num_lapses(t-1)

In [48]:
def num_deaths(t):
    if t < T:
        return num_in_force(t) * q[t]
    else:
        return 0

In [49]:
def num_lapses(t):
    if t < T:
        return num_in_force(t) * w[t]
    else:
        return 0

In [50]:
print(" t : num_in_force(t)")
for t in range(11):
    print(f"{t:2} : {num_in_force(t):0.3f}")

 t : num_in_force(t)
 0 : 1.000
 1 : 0.949
 2 : 0.881
 3 : 0.808
 4 : 0.724
 5 : 0.620
 6 : 0.494
 7 : 0.392
 8 : 0.311
 9 : 0.277
10 : 0.000


We can now calculate the dependent expected cashflows, and an overall cashflow for that time point

In [51]:
def claims(t):
    return num_deaths(t) * S

def premiums(t):
    return num_in_force(t) * P

def net_cashflow(t):
    return premiums(t) - claims(t)

In [52]:
print(" t : claims(t) \t premiums(t) \t net_cashflow(t)")
for t in range(11):
    print(f"{t:4} : {claims(t):8.3f} \t {premiums(t):8.3f} \t {net_cashflow(t):8.3f}")

 t : claims(t) 	 premiums(t) 	 net_cashflow(t)
   0 :   25.000 	  100.000 	   75.000
   1 :   47.450 	   94.900 	   47.450
   2 :   66.050 	   88.067 	   22.017
   3 :   60.568 	   80.758 	   20.189
   4 :   72.440 	   72.440 	    0.000
   5 :   62.008 	   62.008 	    0.000
   6 :   61.698 	   49.359 	  -12.340
   7 :   68.670 	   39.240 	  -29.430
   8 :   70.014 	   31.117 	  -38.897
   9 :   76.245 	   27.726 	  -48.520
  10 :    0.000 	    0.000 	    0.000


## Calculate Present Value

Now we can calculate the present value, for this we would usually use a term dependent yield curve to capture the term dependency, for this example we will assume a flat yield curve, with rate 2% per annum.

In [53]:
def npv(cashflow, term, int_rate):
    v = 1 / (1 + int_rate)
    return sum(cashflow(t) * v**(t+1) for t in range(term))

In [54]:
npv(premiums, T, 0.02)

592.7646738805214

In [55]:
npv(claims, T, 0.02)

542.4398431254847

Finally we can calculate the Expected Net Value, both as an absolute amount and as a percentage margin:

In [56]:
npv(net_cashflow, T, 0.02)

50.32483075503679

In [57]:
margin = npv(net_cashflow, T, 0.02) / npv(premiums, T, 0.02)
print(f"Margin (% of Premium): {margin:0.0%}")

Margin (% of Premium): 8%


So in this case (ignoring expenses), we would expect the policy to be profitable, as the value of premiums received is expected to outweigh the value of claims paid.  The margin (2%) is however fairly low, so introducing expenses are likely to make the policy unprofitable.

## This is a slow computation, how can it be sped up?

 - We can avoid repeated calculations of the same results (such as `num_in_force(5)`) by using a cache/memoization the results from the functions (using a decorator like `@lru_cache` on each result or `heavymodel`)

 - We could compile to c (using cython), or use pypy to perform JIT compilation.  Cython can't compile decorators so memoization needs to be coded separately (e.g. a Python memoization wrapper around the C function).
 
 - We could carry out analysis of the recursive functions and convert these into a non-recursive form.  This is likely to cause more problems than it solves, and if there is conditional logic (e.g. if we take the maximum of two values at a point in time), then it may be impossible.

# Heavy Model approach

In [58]:
# if required: %pip install heavymodel-lewisfogden pyyaml

In [59]:
import heavymodel

## Define the model

I have moved all the functions into the model class, and included the `self` reference so we calculate using the class method which will be cached.  (In practice T, S, P etc would also be attributes of the class, and set on instantiation)

In [60]:
class Term(heavymodel.Model):
    def num_in_force(self, t):
        if t == 0:
            return 1
        elif t >= T:
            return 0
        else:
            return self.num_in_force(t-1) - self.num_deaths(t-1) - self.num_lapses(t-1)
    
    def num_deaths(self, t):
        if t < T:
            return self.num_in_force(t) * q[t]
        else:
            return 0
    
    def num_lapses(self, t):
        if t < T:
            return self.num_in_force(t) * w[t]
        else:
            return 0
        
    def claims(self, t):
        return self.num_deaths(t) * S

    def premiums(self, t):
        return self.num_in_force(t) * P

    def net_cashflow(self, t):
        return self.premiums(t) - self.claims(t)

In [61]:
term = Term()
term._run(T+1)

In [62]:
term._dataframe()

,claims,net_cashflow,num_deaths,num_in_force,num_lapses,premiums
0,25.000000,75.000000,0.001000,1.000000,0.050000,100.000000
1,47.450000,47.450000,0.001898,0.949000,0.066430,94.900000
2,66.050400,22.016800,0.002642,0.880672,0.070454,88.067200
3,60.568217,20.189406,0.002423,0.807576,0.080758,80.757622
4,72.439587,0.000000,0.002898,0.724396,0.101415,72.439587
5,62.008287,0.000000,0.002480,0.620083,0.124017,62.008287
6,61.698245,-12.339649,0.002468,0.493586,0.098717,49.358596
7,68.670147,-29.430063,0.002747,0.392401,0.078480,39.240084
8,70.014120,-38.896733,0.002801,0.311174,0.031117,31.117387
9,76.245377,-48.519785,0.003050,0.277256,0.011090,27.725591


In [63]:
npv(term.net_cashflow, T, 0.02)

50.32483075503679

In [64]:
%timeit term=Term(); term._run(T+1); x = npv(term.net_cashflow, T, 0.02)

171 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [65]:
%timeit x=npv(net_cashflow, T, 0.02)

12.8 ms ± 24 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [67]:
# this result may change
speed_up = 12.8 / (171/1000)
round(speed_up, 2)

74.85

So for this example, we are improving speed by a factor of c. 75x (1000 µs = 1 ms).